In [15]:
#establish STAC search, just like with previous notebooks
#setup band mapping for spyndex
#generate example indices to test process works
#save out geotifs of indices

In [16]:
%pip install spyndex
%pip install cubo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas
import numpy as np
import xarray as xr
import geopandas as gpd
import rasterio
from rasterio.windows import Window
from rasterio.windows import from_bounds
from rasterio import plot
from rasterio.plot import show
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pystac_client

import spyndex
import cubo


In [18]:
def initialize_stac_client(stac_url):
    """
    Initialize and return a STAC client for a given STAC API URL.
    Parameters:
    - stac_url (str): The URL of the STAC API.
    Returns:
    - A pystac_client.Client object
    """
    client = pystac_client.Client.open(stac_url)
    return client


def query_stac_api(client, bbox, collections, start_date=None, end_date=None, limit=10):
    """
    Query a STAC API for items within a bounding box and date range for specific collections.
    Parameters:
    - client: The STAC client initialized with `initialize_stac_client`.
    - bbox (list): The bounding box for the query [min_lon, min_lat, max_lon, max_lat].
    - collections (list): A list of collection IDs to include in the query.
    - start_date (str, optional): The start date for the query (YYYY-MM-DD). Defaults to None.
    - end_date (str, optional): The end date for the query (YYYY-MM-DD). Defaults to None.
    - limit (int): Maximum number of items to return.    
    Returns:
    - A list of STAC Items that match the query parameters.
    """

    search_params = {
        "bbox": bbox,
        "collections": collections,
        "limit": limit
    }
    if start_date and end_date:
        search_params["datetime"] = f"{start_date}/{end_date}"

    search = client.search(**search_params)

    items = list(search.items())
    return items


def inspect_stac_item(item):
    """
    Inspects a STAC item and prints out key information to help identify the data type.

    This function iterates over the assets of a given STAC item, printing details such as the
    asset's title, description, media type, and roles. These details can provide insights into
    the nature of the data contained within the item, such as whether it includes satellite imagery,
    elevation data, or other geospatial resources.

    Parameters:
    - item: A STAC item object. This object should conform to the STAC item specification and
            include properties like an ID, datetime, and a dictionary of assets.

    Returns:
    - None: This function does not return any value. It only prints information to the console.
    """

    # Print the unique identifier of the STAC item
    print("Item ID:", item.id)

    # Print the acquisition date of the data, which is stored in the item's properties
    print("Date:", item.properties.get('datetime'))

    # Begin iterating over the assets associated with this STAC item.
    # Assets represent individual data files or resources related to this item.
    print("Assets:")
    for asset_key, asset in item.assets.items():
        # asset_key is the name used to refer to this asset in the STAC item's assets dictionary.
        # asset is the actual asset object, which contains metadata about the data file or resource.

        # Print the key of the asset and its title. The title is a human-readable name for the asset.
        # If no title is provided, it defaults to 'No title'.
        print(f"  - {asset_key}: {asset.title or 'No title'}")

        # Print a description of the asset, which can provide more context about the data it contains.
        # If no description is provided, it defaults to 'No description'.
        print(f"    Description: {asset.description or 'No description'}")

        # Print the media type of the asset, which indicates the format of the data file (e.g., 'image/tiff' for a GeoTIFF file).
        print(f"    Media Type: {asset.media_type}")

        # Print the roles associated with this asset. Roles are used to describe the function of the asset,
        # such as whether it's the primary data ('data'), metadata about the item ('metadata'), a thumbnail image ('thumbnail'), etc.
        # The roles are joined by a comma in case there are multiple roles.
        print(f"    Roles: {', '.join(asset.roles)}")



In [24]:
geom_fpath = 'input-data/dissolved-boundaries.geojson'
geom = gpd.read_file(geom_fpath)

bbox = list(geom.total_bounds)
lat = geom.centroid.x[0]
long = geom.centroid.y[0]

stac_url_sentinel = "https://earth-search.aws.element84.com/v1"

start_date = "2023-01-01"
end_date = "2023-03-01"

client_sentinel = initialize_stac_client(stac_url_sentinel)

print(lat)
print(long)


/tmp/ipykernel_969/1044281421.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat = geom.centroid.x[0]
/tmp/ipykernel_969/1044281421.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  long = geom.centroid.y[0]


116.32813629361138
-29.263026061552612


In [20]:
items_sentinel = query_stac_api(client_sentinel, bbox, start_date, end_date)

Use the Spyndex package to map satellite bands to parameters, and then calculate whatever indices we want. 

In [21]:
print(spyndex.bands)
# You can also use spyndex to get information about the bands of a specific satellite. 
# In thi case, if I want to know which band should be mapped to the parameter B:
print(spyndex.bands.B.sentinel2a)

['A', 'B', 'G', 'G1', 'N', 'N2', 'R', 'RE1', 'RE2', 'RE3', 'S1', 'S2', 'T', 'T1', 'T2', 'WV', 'Y']
Platform: Sentinel-2A, Band: Blue
        * Band: B2
        * Center Wavelength (nm): 492.4
        * Bandwidth (nm): 66.0
        


In [22]:
#Likewise, you can use spyndex to get information about the spectral index you are interested in:
spyndex.indices.NDVI

SpectralIndex(NDVI: Normalized Difference Vegetation Index)
        * Application Domain: vegetation
        * Bands/Parameters: ['N', 'R']
        * Formula: (N-R)/(N+R)
        * Reference: https://ntrs.nasa.gov/citations/19740022614
        

Testing the 'cubo' package for creating data cubes.
"SpatioTemporal Asset Catalogs (STAC) provide a standardized format that describes geospatial information. Multiple platforms are using this standard to provide clients several datasets. cubo is a Python package that provides users of STAC and GEE an easy way to create On-demand Earth System Data Cubes (ESDCs). This is perfectly suitable for Deep Learning (DL) tasks. You can easily create a lot of ESDCs by just knowing a pair of coordinates and the edge size of the cube in pixels!"

By default cubo uses the Microsoft planetary computer as its endpoint but it does let you specify your own endpoint.

In [23]:
da = cubo.create(
   lat=lat,
   lon=-long,
   collection="sentinel-s2-l2a",
   bands=["B05","B06","B07"],
   start_date="2022-01-01",
   end_date="2022-06-01",
   edge_size=128,
   resolution=10,
   stac="https://earth-search.aws.element84.com/v1"
)

IndexError: list index out of range